In [49]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import torch

In [50]:
from fastai.vision import *

In [51]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
from sklearn.datasets import load_boston
from torch import nn,optim
import torch as th
import syft as sy

In [52]:
hook = sy.TorchHook(th)
boston = load_boston()
boston_data = pd.DataFrame(boston.data)


In [53]:
print(boston.keys())


dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])


In [54]:
bo = pd.DataFrame(boston.data)
bo.columns = boston.feature_names
print(bo.head())
boston_target = boston.target

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  
0     15.3  396.90   4.98  
1     17.8  396.90   9.14  
2     17.8  392.83   4.03  
3     18.7  394.63   2.94  
4     18.7  396.90   5.33  


In [55]:
print(boston_target)

[24.  21.6 34.7 33.4 ... 20.6 23.9 22.  11.9]


In [56]:
X=torch.tensor(boston_data.values,requires_grad=True,dtype=torch.float)
Y=torch.tensor(boston_target,requires_grad=True,dtype=torch.float)

In [57]:
print (X.shape)

torch.Size([506, 13])


In [58]:
Y=Y.reshape((506,1))

In [59]:
bob=sy.VirtualWorker(hook,id='bob')


In [60]:
bill=sy.VirtualWorker(hook,id='bill')
rupert = sy.VirtualWorker(hook,id='rupert')

In [61]:
bob_train= X[0:180].send(bob)
bob_target=Y[0:180].send(bob)

In [62]:
bill_train = X[180:360].send(bill)
bill_target=Y[180:360].send(bill)
test_data = X[360:]
test_target = Y[360:].send(rupert)

In [63]:
loss_function= nn.MSELoss()

In [64]:
loss_function_bob = nn.MSELoss()

In [65]:
datasets =[(bob_train,bob_target),(bill_train,bill_target)]

In [66]:
def federated(iterations=100):

    model=nn.Sequential(nn.Linear(13, 18), nn.ReLU(),
                       nn.Linear(18, 10), nn.ReLU(),
                       nn.Linear(10, 5), nn.ReLU(),
                       nn.Linear(5, 1))
    opt =optim.SGD(params=model.parameters(),lr=0.01)
    
    for iter in range (iterations):
   
        for _data,_target in datasets:
             # sending model
                model=model.send(_data.location)

             #training on the different machines
                opt.zero_grad()
                pred=model(_data)
                loss=loss_function(pred,_target)                                 
                loss.backward()
                opt.step()
             
             #Get trained model
                model=model.get()

                print(loss.get())


In [67]:
federated()

tensor(595.3028, requires_grad=True)
tensor(2803.2136, requires_grad=True)
tensor(462.5965, requires_grad=True)
tensor(746.7130, requires_grad=True)
tensor(425.7882, requires_grad=True)
tensor(700.4594, requires_grad=True)
tensor(392.0512, requires_grad=True)
tensor(657.5869, requires_grad=True)
tensor(361.1385, requires_grad=True)
tensor(617.8418, requires_grad=True)
tensor(332.8227, requires_grad=True)
tensor(580.9890, requires_grad=True)
tensor(306.8945, requires_grad=True)
tensor(546.8116, requires_grad=True)
tensor(283.1609, requires_grad=True)
tensor(515.1093, requires_grad=True)
tensor(261.4444, requires_grad=True)
tensor(485.6970, requires_grad=True)
tensor(241.5815, requires_grad=True)
tensor(458.4035, requires_grad=True)
tensor(223.4216, requires_grad=True)
tensor(433.0713, requires_grad=True)
tensor(206.8262, requires_grad=True)
tensor(409.5540, requires_grad=True)
tensor(191.6677, requires_grad=True)
tensor(387.7168, requires_grad=True)
tensor(177.8286, requires_grad=True)
